# Demo Notebook

Only the necessary code for the presentation.

In [3]:
# Imports
from src import model, thymio, vision, pathfinder, navglobal, navlocal, filtering, draw
import numpy as np

In [ ]:
# Use the webcam, adapt the index according to your computer and the webcamp port
# This index corresponds to OpenCV's VideoCapture index
source = vision.WebcamSource(index=2)
# Create our vision tools
tools = vision.VisionTools()

In [ ]:
# Calibrate with the aruco markers, this can take a few frames
ready = False
while not ready:
    img = source.get_frame()
    corners_by_id = tools.get_aruco_dict(img)
    try:
        calibrated = tools.get_aruco_calibrated(img, corners_by_id)
        ready = True
    except:
        ready = False
draw.plot_image(calibrated)

In [ ]:
# Pick the right blue bounds
blue_bounds=vision.HSVBound(lb=np.array([80, 60, 120]), ub=np.array([100, 120, 150]))
mask = tools.get_color_mask(calibrated, blue_bounds)
draw.plot_image(mask)

In [ ]:
# Pick the right red bounds
red_bounds=vision.HSVBound(lb=np.array([170, 100, 130]), ub=np.array([180, 180, 250]))
mask = tools.get_color_mask(calibrated, red_bounds)
draw.plot_image(mask)

In [ ]:
# Our vision pipeline is ready to be used!
viz = vision.VisionPipeline(source, tools=tools, red_bounds=red_bounds, blue_bounds=blue_bounds, dilate_factor=140)
# Compute the world
world = viz.analyze_scene()
# Compute the path
path = pathfinder.find_path(world, tools.target_resolution)
# Display result
result = draw.draw_world(calibrated, world)
result = draw.draw_path(result, path)
draw.plot_image(result)

In [ ]:
# Connect with the Thymio
th = thymio.Thymio()
# Setup global and local navigation object
navigator = navglobal.GlobalNavigation(path)
avoider = navlocal.LocalNavigation()
# Flag to check if we were in local avoidance
# This is used to recompute the path and update global navigation
was_in_avoidance = False
# Create the filter with initial state
kalman = filtering.Filter(world.robot)

# While we have not reached the goal
while not world.robot_at_goal(60):
    # Read data from the sensors
    sensor_data = th.read_sensor_data()

    # Check whether to use local or global navigation
    if avoider.avoidance_mode(sensor_data):
        command = avoider.next_command(sensor_data)
        was_in_avoidance = True
    else:
        # Recompute path if we left local avoidance
        if was_in_avoidance:
            path = pathfinder.find_path(world, tools.target_resolution)
            navigator = navglobal.GlobalNavigation(path)
            was_in_avoidance = False
        command = navigator.next_command(world.robot)
    # Send the command to the Thymio
    th.process_command(command)
    # Read the pose from the camera
    pose = viz.get_robot_pose()
    # Update state with the filter
    world.robot = kalman.update_robot(viz.last_robot_pose, command, sensor_data, camera=pose is not None)
# Disconnect the Thymio
th.stop()

# Show the last frame
draw.plot_image(viz.latest_frame)